In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from math import sin, cos, sqrt, atan2, radians
from geopy.distance import vincenty
from sklearn import preprocessing
from scipy import interpolate
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
import networkx as nx
# First, import the data:
pd.options.display.max_columns = 100
# pd.options.display.max_rows = None


## Adding a month of data - Jan 2016 - Feb 2016


list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116']
#                     'fahrzeiten_soll_ist_20160117_20160123', 'fahrzeiten_soll_ist_20160124_20160130']
#                     'fahrzeiten_soll_ist_20160131_20160206', 'fahrzeiten_soll_ist_20160207_20160213',
#                     'fahrzeiten_soll_ist_20160214_20160220', 'fahrzeiten_soll_ist_20160221_20160227',
#                     'fahrzeiten_soll_ist_20160228_20160305', 'fahrzeiten_soll_ist_20160306_20160312',
#                     'fahrzeiten_soll_ist_20160313_20160319', 'fahrzeiten_soll_ist_20160320_20160326']

# Just the first two weeks

# list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116']

list_of_df = []
for i in list_of_df_names:
    list_of_df.append(pd.read_csv(i+'.csv'))
df_1 = pd.concat(list_of_df, ignore_index=True)
list_of_df.clear()


####################
df_haltepunkt = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltepunkt.csv")
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
# df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
# df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


# Bus Station
df_haltestelle = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltestelle.csv")
####################

def find_dist(lat1, lon1, lat_2, lon2):
    c1 = (lat1, lon1)
    c2 = (lat_2, lon2)
    return geopy.distance.vincenty(c1, c2).km
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################


In [2]:
# Drop the punkts with undefined gps data
df_haltepunkt.dropna(axis=0, how='any', inplace=True, subset=['GPS_Longitude'])


In [3]:
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


In [4]:
# Number of records
len(df_1)

2804468

In [5]:
# Merge the halt punkt from


df_1['halt_punkt_id'] = df_1['halt_punkt_id_von']
df_1 = df_1.merge(df_haltepunkt, on=['halt_punkt_id'])


In [6]:
# df_1['GPS_Latitude_from'] = df_1['GPS_Latitude']
# df_1['GPS_Longitude_to'] = df_1['GPS_Longitude']

# Merge punkt to

df_1['halt_punkt_id'] = df_1['halt_punkt_id_nach']
df_1 = df_1.merge(df_haltepunkt, on='halt_punkt_id')


In [7]:

df_1['halt_id'] = df_1['halt_id_x']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [8]:
df_1['halt_id'] = df_1['halt_id_y']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2804468 entries, 0 to 2804467
Data columns (total 56 columns):
linie                     int64
richtung                  int64
betriebsdatum             object
fahrzeug                  int64
kurs                      int64
seq_von                   int64
halt_diva_von             int64
halt_punkt_diva_von       int64
halt_kurz_von1            object
datum_von                 object
soll_an_von               int64
ist_an_von                int64
soll_ab_von               int64
ist_ab_von                int64
seq_nach                  int64
halt_diva_nach            int64
halt_punkt_diva_nach      int64
halt_kurz_nach1           object
datum_nach                object
soll_an_nach              int64
ist_an_nach1              int64
soll_ab_nach              int64
ist_ab_nach               int64
fahrt_id                  int64
fahrweg_id                int64
fw_no                     int64
fw_typ                    int64
fw_kurz           

In [10]:
# delete the entries where the days are not matching
df_1 = df_1[df_1['betriebsdatum'] == df_1['datum_nach']]

In [11]:
# Some Signals
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'
delay = 'delay'

actual_stationary = 'actual_stationary'
actual_travel_time = 'actual_travel_time'

df_1[target_stationary] = df_1[target_departure_from] - df_1[target_arrival_from]
df_1[target_travel_time] = df_1[target_arrival_to] - df_1[target_departure_from]
df_1[delay] = df_1[actual_arrival_from] - df_1[target_arrival_from]


# Signal - 1 - actual arrival - actual departure
df_1[actual_stationary] = df_1[actual_departure_from] - df_1[actual_arrival_from]
df_1[actual_travel_time] = df_1[actual_arrival_to] - df_1[actual_departure_from]

# b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
# b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

# Merge halts - Define some attr.

In [12]:
# Creating a pair of gps locations and calculating the signal at that time instance:
# temp['Road_Segment_from'] = list(zip(temp.GPS_Latitude_x, temp.GPS_Longitude_x))
# temp['Road_Segment_to'] = list(zip(temp.GPS_Latitude_y, temp.GPS_Longitude_y))
# temp

# aggregating the gps locations of the same punkts
df_new_x = df_1.groupby(['halt_id_x'], as_index=False)['GPS_Latitude_x', 'GPS_Longitude_x'].mean()
df_new_y = df_1.groupby(['halt_id_y'], as_index=False)['GPS_Latitude_y', 'GPS_Longitude_y'].mean()
# Merging the two frames signifying a road segment
final = df_1.merge(df_new_x, on=['halt_id_x'])
final = final.merge(df_new_y, on=['halt_id_y'])


In [13]:
final.rename(columns={'GPS_Latitude_x_y':'GPS_LAN_STELLE_FROM', 'GPS_Longitude_x_y':'GPS_LON_STELLE_FROM',
                      'GPS_Latitude_y_y':'GPS_LAN_STELLE_TO', 'GPS_Longitude_y_y':'GPS_LON_STELLE_TO'}, inplace=True)
# final

In [14]:
# To release memeory
df_1 = 1

In [15]:
# If This treats the roads as punkt to punkt
# final['Road_Segment_From'] = list(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x))
# final['Road_Segment_To'] = list(zip(final.GPS_Latitude_y_x, final.GPS_Longitude_y_x))
# final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
# final['Route_Node_id_tuple'] = list(zip(final.halt_punkt_id_von, final.halt_punkt_id_nach))


## The following treats the road segments as haltestelle to haltestelle
final['Road_Segment_From'] = list(zip(final.GPS_LAN_STELLE_FROM, final.GPS_LON_STELLE_FROM))
final['Road_Segment_To'] = list(zip(final.GPS_LAN_STELLE_TO, final.GPS_LON_STELLE_TO))
final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
final['Route_Node_id_tuple'] = list(zip(final.halt_id_x, final.halt_id_y))
final.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple
0,918,1,06.01.16,11652,1,2,3744,50,ZENK,06.01.16,65880,65906,65898,65906,3,2043,50,RKIK,06.01.16,65988,65997,66000,65997,75165,5091,1,1,1,BKUE - GART,112395,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,26,0,91,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)"
1,918,1,06.01.16,11664,2,2,3744,50,ZENK,06.01.16,24480,24561,24492,24572,3,2043,50,RKIK,06.01.16,24576,24657,24588,24657,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,81,11,85,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)"
2,918,1,09.01.16,11655,2,2,3744,50,ZENK,09.01.16,53280,53313,53292,53325,3,2043,50,RKIK,09.01.16,53376,53407,53388,53407,139140,5091,1,1,1,BKUE - GART,113211,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,33,12,82,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)"
3,918,1,08.01.16,11653,2,2,3744,50,ZENK,08.01.16,24480,24510,24492,24510,3,2043,50,RKIK,08.01.16,24576,24586,24588,24586,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,30,0,76,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)"
4,918,1,08.01.16,11646,2,2,3744,50,ZENK,08.01.16,53280,53319,53292,53334,3,2043,50,RKIK,08.01.16,53376,53434,53388,53445,75211,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,39,15,100,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)"


In [16]:
# Drop some unwanted columns
columns_to_drop = ['halt_diva_von', 'halt_punkt_diva_von', 'halt_diva_nach', 'halt_punkt_diva_nach', 'fw_kurz']
final.drop(columns=columns_to_drop, inplace=True)

# Addressing the signal intervals:

    Dealing with granularity issues:

In [17]:
# reformat the date
d = pd.to_datetime(final['betriebsdatum'], format='%d.%m.%y', dayfirst=True)
# Monday == 0 … Sunday == 6
final['day_of_week'] = [i.weekday() for i in d]

In [18]:
# Remove type 2 - irrelevant 
final = final[final.fw_typ != 2]

In [19]:
# Make sure the GPS data is in order
list(set(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x)))[:5]

[(47.30712, 8.548644),
 (47.409068, 8.595832),
 (47.38303, 8.499871),
 (47.358689, 8.527014),
 (47.370287, 8.514156)]

In [20]:
# # encode routes for easier looping
final['Route_Node_id_str'] = final.Route_Node_id_tuple.apply(lambda x: '{}_{}'.format(x[0], x[1]))
le = preprocessing.LabelEncoder()
le.fit(final.Route_Node_id_str)
final['Route_Node_id'] = le.transform(final.Route_Node_id_str)

In [21]:
# Drop Sat Sun
final = final[((final.day_of_week != 5) & (final.day_of_week != 6))]
final = final[(final['soll_an_von'] >= 21600) & (final['soll_an_von'] <= 72000)]

In [22]:
# Total road segments
len(set(final.Road_Segment_Node))

1560

In [23]:
final.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple,day_of_week,Route_Node_id_str,Route_Node_id
0,918,1,06.01.16,11652,1,2,ZENK,06.01.16,65880,65906,65898,65906,3,RKIK,06.01.16,65988,65997,66000,65997,75165,5091,1,1,BKUE - GART,112395,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,26,0,91,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)",2,2663_2796,1200
1,918,1,06.01.16,11664,2,2,ZENK,06.01.16,24480,24561,24492,24572,3,RKIK,06.01.16,24576,24657,24588,24657,75179,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,81,11,85,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)",2,2663_2796,1200
3,918,1,08.01.16,11653,2,2,ZENK,08.01.16,24480,24510,24492,24510,3,RKIK,08.01.16,24576,24586,24588,24586,75179,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,30,0,76,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)",4,2663_2796,1200
4,918,1,08.01.16,11646,2,2,ZENK,08.01.16,53280,53319,53292,53334,3,RKIK,08.01.16,53376,53434,53388,53445,75211,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,39,15,100,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)",4,2663_2796,1200
5,918,1,08.01.16,11653,2,2,ZENK,08.01.16,42480,42497,42492,42508,3,RKIK,08.01.16,42576,42591,42588,42591,75199,5091,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.31823,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,17,11,83,47.318319,8.581964,47.316607,8.583933,"(47.31831928797488, 8.581964336234076)","(47.31660700000026, 8.58393299999993)","((47.31831928797488, 8.581964336234076), (47.3...","(2663, 2796)",4,2663_2796,1200


In [24]:
# Define the distance signal based on punkt to punkt
# from geopy.distance import geodesic
# # geodesic(newport_ri, cleveland_oh).miles
# final['distance'] = final.apply(lambda x: vincenty((x['GPS_Latitude_x_x'], x['GPS_Longitude_x_x']), (x['GPS_Latitude_y_x'], x['GPS_Longitude_y_x'])).meters, axis=1)

In [25]:
# final.distance.isna().sum()

In [26]:
# final['speed'] = final.distance/final.actual_travel_time

In [27]:
# Define the time intervals
time_period = 600
# Previously was at 5 am
starting_time = 21600
# indicates 10 mins intervals
# Number of bins:
n_bins = int(np.floor((72000 - starting_time)/time_period))
labels = [i for i in range(n_bins)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins + 1)]
final['intervals'] = pd.cut(final[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels, right=True, include_lowest=True)

In [28]:
len(labels)

84

In [29]:
final.Route_Node_id.value_counts()[:5]

1058    5526
205     5502
139     4572
377     4569
295     4564
Name: Route_Node_id, dtype: int64

In [30]:
# final['actual_travel_time_n_target'] = final.actual_travel_time / final.target_travel_time
# final.actual_travel_time_n_target = final.actual_travel_time_n_target - final.actual_travel_time_n_target.mean()
# final.actual_travel_time_n_target_mean = final.actual_travel_time_n_target_mean - final.actual_travel_time_n_target_mean.mean()

# Define new signal: Normalized travel time - close to TTI - Version 1, normalize by target travel time

In [31]:
final['actual_travel_time_n_target'] = final.actual_travel_time.div(final.target_travel_time, axis=0)
# final

# Define new signal: Normalized travel time - Version 2, normalize by mean of target in that interval

In [32]:
final['actual_travel_time_n_target_mean'] = final.actual_travel_time.div(final.target_travel_time.mean(), axis=0)
# final

In [33]:
%matplotlib
a = final[final.Route_Node_id == 1067]
a = a[a.betriebsdatum == '08.01.16']
# a = a.sort_values(by=target_departure_from)
a = a.groupby(by='intervals', as_index=False).mean()
# a.intervals.value_counts()
# a = a.groupby(by='intervals', as_index=False).mean()
# a.fillna(value=0)
# plt.plot(a[target_departure_from], a.target_travel_time)
plt.plot(a.intervals.values.tolist(), a.actual_travel_time_n_target.values.tolist(), a.intervals.values.tolist(), a.actual_travel_time_n_target.values.tolist())

Using matplotlib backend: Qt5Agg


In [34]:
a = final[final.Route_Node_id == 1067]
a = a[a.betriebsdatum == '08.01.16']
for i in set(a.fahrweg_id):
    b = a[a.fahrweg_id == i]
    b.sort_values(by=['intervals', target_departure_from], inplace=True)
    if b.shape[0] < 50:
        continue
    plt.plot(b[target_departure_from], b.actual_travel_time_n_target)
a = a.groupby(by='intervals', as_index=False).mean()
plt.title('Signal')
plt.xlabel('Times of the day')
plt.ylabel('')
a.sort_values(by=['intervals', target_departure_from], inplace=True)
plt.scatter([time_period * i + 21600 for i in a.intervals.values.tolist()], np.array(a.actual_travel_time_n_target.values.tolist()), marker='^', s=1000)

In [35]:
# Road seg in the mid in terms of how many recordings there are
final.Route_Node_id.value_counts()[200:205]

148    2216
547    2216
195    2215
105    2214
439    2214
Name: Route_Node_id, dtype: int64

In [36]:
# Find the Nodes with gran. problems:

list_road_seg = final.Route_Node_id.value_counts()
list_road_seg_id = []
list_empty_bins_index = []
list_to_drop = []

for index, item in list_road_seg.items():
    temp = final[final.Route_Node_id == index]
    if (0 in (temp.intervals.value_counts().value_counts())):
        list_road_seg_id.append(index)
        list_empty_bins_index.append(temp.intervals.value_counts().value_counts()[0])
        list_to_drop.append(index)
    else:
        list_road_seg_id.append(index)
        list_empty_bins_index.append(0)
list_empty_bins_index = np.array(list_empty_bins_index)

In [37]:
# Number of road segments where they have some empty bins
len(set(list_to_drop))

838

In [38]:
# How many are actually empty
# list_empty_bins_index

In [39]:
%matplotlib
len(list_empty_bins_index)
pd.Series(list_empty_bins_index).hist()
plt.title('Distribution of empty and non empty bins - 10 mins - Start at 7 - 8')
plt.xlabel('Road ID')
plt.ylabel('Number of empty bins')

Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Number of empty bins')

In [40]:
# %matplotlib
# # Plot the halts vs. empty bins
# plt.scatter(list_road_seg_id, list_empty_bins_index)
# plt.title('Scatter of empty and non empty bins - 10 mins  - Start at 7 - 10')
# plt.xlabel('Road ID')
# plt.ylabel('Number of empty bins')


In [41]:
# How many missing below 6 - how many exactly 0 
print(np.count_nonzero(list_empty_bins_index < 6))
print(np.count_nonzero(list_empty_bins_index == 0))

798
722


# A Clear Issue with granularity
solving it by limiting the start date and visualizing the distribution


In [42]:
# rev_dist_bins = pd.Series(index = final.intervals.value_counts().values, data=final.intervals.value_counts().index.values)
plt.scatter(final.intervals.value_counts().index.values, final.intervals.value_counts().values)
plt.title('scatter - bin number vs data count - start at 7, end at 12 - 10 mins')
plt.xlabel('Bin number')
plt.ylabel('Count of Data points')

Text(0, 0.5, 'Count of Data points')

In [43]:
# top bins with the most number of recordings
final.intervals.value_counts()[:5]

10    23156
7     23078
6     22900
11    22834
9     22791
Name: intervals, dtype: int64

In [44]:
# Drop those datapoints:
print(np.count_nonzero(list_empty_bins_index != 0))
len(list_to_drop)

838


838

In [46]:
len(final)

1708076

In [47]:
final = final[~final.Route_Node_id.isin(list_to_drop)]

In [48]:
len(final)

1260431

In [50]:
# most recordings per route
final.Route_Node_id.value_counts()[:5]

1058    5526
205     5502
139     4572
377     4569
295     4564
Name: Route_Node_id, dtype: int64

# Create Signal

In [51]:
# Creating Signals for all of the nodes:
List_of_Nodes = [str(i) for i in list(set(final.Route_Node_id))]

In [52]:
len(List_of_Nodes)

722

In [53]:
List_of_head_data = {}
for i in List_of_Nodes:
    temp_df = final[final.Route_Node_id == int(i)]
#     temp_df = temp_df.sort_values(by=['betriebsdatum'])
    time_series_for_node = []
    for j in sorted(list(set(temp_df.betriebsdatum)), key=lambda x: (x[3], x[4], x[0], x[1])):
#         print(j)
        time_series = temp_df[temp_df.betriebsdatum == j]
        time_series = time_series.groupby(by='intervals', as_index=False).mean()
        time_series = time_series['actual_travel_time_n_target']
        time_series_for_node.append(time_series)
    final_series = pd.concat(time_series_for_node, ignore_index=True)
    List_of_head_data[i] = final_series

In [54]:
# count nans in the signals
new_dict = {}
for key, value in List_of_head_data.items():
    new_dict[key] = value.isna().sum()

In [55]:
new_dict

{'0': 2,
 '1': 1,
 '7': 0,
 '10': 0,
 '11': 0,
 '12': 0,
 '13': 1,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '19': 0,
 '20': 1,
 '21': 17,
 '22': 14,
 '23': 4,
 '24': 0,
 '25': 3,
 '26': 0,
 '27': 0,
 '30': 1,
 '31': 23,
 '33': 1,
 '36': 2,
 '37': 4,
 '38': 0,
 '39': 1,
 '40': 0,
 '42': 0,
 '43': 1,
 '54': 2,
 '55': 0,
 '56': 2,
 '57': 4,
 '58': 0,
 '59': 2,
 '65': 0,
 '66': 0,
 '72': 4,
 '74': 0,
 '75': 2,
 '76': 4,
 '77': 2,
 '78': 7,
 '85': 1,
 '86': 0,
 '88': 0,
 '91': 3,
 '92': 1,
 '93': 32,
 '94': 20,
 '95': 0,
 '96': 0,
 '97': 0,
 '98': 24,
 '99': 29,
 '100': 0,
 '101': 9,
 '102': 0,
 '104': 0,
 '105': 0,
 '106': 0,
 '109': 0,
 '115': 22,
 '116': 0,
 '117': 20,
 '128': 3,
 '129': 3,
 '133': 0,
 '135': 4,
 '139': 3,
 '140': 0,
 '141': 0,
 '142': 0,
 '144': 0,
 '145': 0,
 '146': 0,
 '147': 0,
 '148': 0,
 '152': 0,
 '153': 0,
 '156': 1,
 '157': 0,
 '159': 0,
 '160': 0,
 '161': 23,
 '166': 4,
 '169': 0,
 '170': 0,
 '171': 0,
 '172': 0,
 '173': 9,
 '174': 6,
 '177': 0,
 '178': 5,
 '17

In [57]:
# Finding dicts with distribution
nan_dist = pd.Series(new_dict)
nan_dist.hist()

In [59]:
# find and drop ones with 15 or more missing values
final_data_df = pd.DataFrame.from_dict(List_of_head_data)
print(sorted(final_data_df.isna().sum()))
final_data_df.dropna(thresh=len(final_data_df) - 15, axis=1, inplace=True)
# How many nodes are left?
print(len(final_data_df.columns))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [66]:
# How the nans are distributed
final_data_df.isna().sum().hist()

In [69]:
# How many are truly zero
final_data_df.isna().sum().value_counts()[0]

298

In [60]:
print(sorted(final_data_df.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [70]:
# How many total nans are present
final_data_df.isna().sum().sum()

1512

In [71]:
final_data_df.head()

,0,1,7,10,11,12,13,14,15,16,17,19,20,22,23,24,25,26,27,30,33,36,37,38,39,40,42,43,54,55,56,57,58,59,65,66,72,74,75,76,77,78,85,86,88,91,92,95,96,97,...,1461,1462,1464,1466,1467,1468,1473,1476,1477,1482,1483,1484,1485,1486,1487,1488,1490,1492,1493,1499,1500,1503,1506,1507,1508,1509,1510,1511,1512,1523,1524,1539,1540,1542,1543,1544,1545,1548,1554,1556,1571,1572,1573,1574,1575,1576,1577,1578,1580,1581
0,1.152778,0.907407,1.083333,1.158333,1.378472,1.083333,1.058333,0.870370,1.053613,1.064103,1.016667,1.041667,1.089744,1.031250,0.500000,1.036710,1.015625,1.060606,0.883838,0.759259,1.100000,0.981481,0.904762,1.195718,0.941667,1.194246,0.972222,0.916667,0.981481,1.045455,1.097222,1.222222,1.055556,1.094792,0.811111,1.070370,0.939394,1.095238,0.916667,1.048611,1.000000,1.133333,1.380952,0.642857,1.737179,1.055556,1.194444,1.069444,1.500000,0.925439,...,1.472222,0.533333,0.966667,0.944444,1.069444,1.166667,1.104167,1.118056,1.142857,0.966667,1.333333,1.333333,1.047619,1.750000,0.933333,0.979167,0.991667,1.208333,1.022222,1.143939,1.033333,1.125000,1.208333,1.208333,1.203704,0.916667,1.166667,1.051282,0.906250,0.935897,1.104167,0.850000,1.159091,1.046732,1.050926,1.022222,1.089744,0.903846,1.026042,0.947917,1.181818,1.089744,1.074561,NaN,0.974116,1.182540,1.027778,1.092593,1.283333,1.100000
1,1.027778,0.962963,1.178571,1.038889,1.150000,0.883333,1.041667,0.919753,1.258159,1.307692,0.975000,0.909722,0.952991,1.005208,0.777778,1.070228,1.145833,1.020202,0.954545,0.851852,1.231250,0.981481,0.857143,1.262108,1.099242,1.198942,0.967593,0.875000,0.990741,1.181818,1.083333,1.259259,0.965278,1.033333,0.891667,0.911111,0.946970,1.166667,1.000000,1.027778,1.050505,1.200000,1.380952,0.571429,1.634615,1.069444,1.166667,1.333333,0.914141,0.905702,...,1.083333,1.033333,1.066667,0.981481,0.972222,1.283333,0.958333,0.948864,1.081845,1.033333,1.064815,1.089286,1.058532,1.550000,0.966667,1.010417,0.916667,0.972222,0.950000,1.121212,1.133333,0.986111,1.064815,1.062500,1.046296,0.941667,1.074074,1.247863,0.875000,1.243590,1.416667,0.825000,1.045455,1.041830,0.979167,0.966667,1.070513,0.961538,1.004464,0.986111,NaN,NaN,NaN,1.063725,1.018519,1.293651,0.916667,1.222222,1.133333,1.116667
2,1.108333,1.268519,1.023810,0.973611,1.044271,1.105556,1.400000,0.907407,0.946096,1.243590,1.000000,0.937500,0.948718,1.145833,0.666667,1.106250,1.031250,1.111111,0.969697,1.129630,0.951389,1.055556,1.273810,1.107550,1.045455,1.135913,0.972222,1.027778,1.041667,1.166667,1.097222,1.250000,1.361111,NaN,0.873333,0.977778,1.000000,1.077381,1.047619,NaN,1.166667,1.133333,1.190476,0.690476,1.649573,1.097222,1.138889,0.939815,1.030303,0.978070,...,1.361111,0.933333,0.933333,0.953704,0.944444,1.233333,0.666667,1.048401,1.026786,0.983333,1.018519,0.857143,0.916667,1.016667,1.125000,1.270833,1.033333,1.006944,1.025926,1.045455,1.233333,1.166667,1.080247,1.009259,1.018519,1.033333,1.166667,1.025641,0.958333,0.935897,1.281250,0.850000,0.992424,1.115033,1.018519,1.054630,1.102564,1.012821,1.015811,0.965278,1.242424,1.038462,1.000000,0.960784,1.116162,1.214286,0.972222,1.203704,1.191667,1.150000
3,1.123148,1.000000,1.310185,1.075000,1.041667,1.133333,0.833333,0.972222,1.043706,1.034188,0.927778,0.854167,1.012821,1.062500,0.611111,1.060820,0.945216,1.121212,0.858586,0.916667,1.006250,0.981481,1.047619,1.372507,1.002020,1.150476,0.916667,1.013889,1.018519,1.083333,1.090278,1.212963,1.004630,0.966667,0.877778,1.113889,1.090909,1.011905,0.988095,1.060185,1.040404,1.179630,1.119048,0.714286,1.634615,0.958333,1.152778,1.300926,0.924242,0.899123,...,1.277778,1.066667,0.933333,1.018519,1.129630,1.483333,0.895833,1.013889,1.081845,1.000000,1.000000,1.277778,1.122024,1.302399,0.883333,1.166667,1.033333,0.958333,1.034722,1.234848,1.025000,1.104167,1.199074,0.870370,1.055556,0.933333,1.101852,0.965812,0.902778,0.955556,1.479167,0.783333,0.909091,1.140087,1.046296,1.050926,0.980769,1.384615,1.041915,1.027778,1.181818,1.025641,0.991228,1.107843,1.031566,1.190476,1.020833,1.055556,1.133333,1.125000
4,1.

In [79]:
# Interpolate the missing values
final_data_df = final_data_df.interpolate(axis=1, limit=16)

In [101]:
# How many nans are left
nan_counts = final_data_df.isna().sum().sum()
nan_counts

2

In [106]:
if nan_counts:
    # drop the column
    final_data_df.dropna(how='any', axis=1)
    # Find which column has the nan
    toDrop = final_data_df[final_data_df.isna() == True].index[0]
    print(toDrop)
    final_data_df.drop(str(toDrop), axis=1, inplace=True)
    
    

0


In [108]:
# Make sure it is dropped
final_data_df.isna().sum().sum()

0

In [114]:
# Visualize some of the signals
final_data_df.columns.values.tolist()[20:25]

['36', '37', '38', '39', '40']

In [118]:
sig_1 = final_data_df['39']
sig_2 = final_data_df['40']

In [119]:
plt.plot(final_data_df.index, sig_1, final_data_df.index, sig_2)

# Signal is Defined, Create the graphs in two ways! Using the signal DF!

# Create The graph - V1

In [122]:
final['Mid_point_GPS_data'] = final.Road_Segment_Node.apply(lambda x: (((x[0][0] + x[1][0])/2) , ((x[0][1] + x[1][1])/2)))

In [124]:
# How many nodes left
len(final_data_df.columns.values.tolist())

627

In [126]:
# List of road segments by id
list_of_nodes_Road = list(set(final_data_df.columns.values.tolist()))

In [127]:
from geopy.distance import vincenty
from geopy.distance import geodesic
from networkx.drawing.nx_pydot import write_dot

In [128]:
# Edit final to get the most updated gps data
list_of_nodes_Road_toKeep = [int(i) for i in list_of_nodes_Road]
final = final[final.Route_Node_id.isin(list_of_nodes_Road_toKeep)]
# final

In [130]:
# Create List of dicts:

new_final = final.groupby(by='Route_Node_id', as_index=False).min()

In [133]:
Dict_List = pd.Series(new_final.Mid_point_GPS_data.values,index=new_final.Route_Node_id).to_dict()
Dict_List

{1: (47.3692079778898, 8.504036502312001),
 7: (47.36850198423427, 8.49862491923999),
 10: (47.36875235731088, 8.49352914745619),
 11: (47.36872171146885, 8.546980809299008),
 12: (47.37150489650463, 8.54966933419556),
 13: (47.36902609339306, 8.550574737329114),
 14: (47.37161366759673, 8.548132660779444),
 15: (47.36985046591957, 8.551908632398236),
 16: (47.37380208933983, 8.52759920903382),
 17: (47.37427443482434, 8.523350822392757),
 19: (47.37871462072661, 8.548135911210938),
 20: (47.37839180232784, 8.546362316672727),
 22: (47.376119412289725, 8.550366966055673),
 23: (47.39832264789874, 8.533202103940999),
 24: (47.398057656056515, 8.5375123534115),
 25: (47.39713015457202, 8.532690477937614),
 26: (47.4002022655336, 8.534157057095538),
 27: (47.39664498928716, 8.534777521252654),
 30: (47.39813144539022, 8.530991834217213),
 33: (47.391890639563215, 8.47689337266727),
 36: (47.39132496239834, 8.481511696175103),
 37: (47.389901155032305, 8.481056399389612),
 38: (47.41069188

In [134]:
# Find edges and weights:
list_of_edges = []

# iterate over each mid point aka road segment
for i in list_of_nodes_Road:
    # Look at every other road segment, if physical distance was less than 2.5 kilometers, add an edge
    
    for key, value in Dict_List.items():
        if Dict_List[int(i)] != value:
            if vincenty(Dict_List[int(i)], value).meters < 1000:
                list_of_edges.append((int(i), key, 10/(vincenty(Dict_List[int(key)], Dict_List[int(i)]).kilometers)))
    
#     a = new_final[new_final.Route_Node_id == int(i)]
#     a.Route_Node_id_tuple.iloc[5]
#     for j in list_of_nodes:
#         if i[0] != j[0]:
#             if vincenty(i[1], j[1]).meters < 1000:
#                 list_of_edges.append((i[0], j[0]))

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # This is added back by InteractiveShellApp.init_path()


In [135]:
# Edge count by the node id
zlst = list(zip(*list_of_edges))
edge_series = pd.Series(zlst[1], index = zlst[0])
edge_series.value_counts()

339     82
1512    82
1367    82
1160    82
335     81
452     81
340     81
1355    81
446     80
334     80
182     80
178     80
378     80
1356    80
177     80
1366    80
451     80
295     80
179     80
253     79
449     79
529     79
185     79
450     79
1338    79
540     79
538     78
528     78
186     78
336     78
        ..
1580     7
862      7
1503     7
228      7
75       7
1581     7
227      7
259      7
244      7
1452     7
360      6
1573     6
54       6
101      6
875      6
1007     5
876      5
359      5
860      5
166      5
116      5
389      5
436      5
894      5
1571     4
1574     4
1572     4
665      3
850      1
221      1
Length: 626, dtype: int64

In [136]:
# Define Graph
G = nx.Graph()

In [137]:
G.add_nodes_from(list_of_nodes_Road_toKeep)

In [138]:
len(set(list_of_edges))

23668

In [139]:
list_of_edges[:5]

[(1407, 927, 10.209181208514154),
 (1407, 943, 15.008160388182691),
 (1407, 948, 10.209471681742789),
 (1407, 951, 22.043982251842735),
 (1407, 952, 10.074790022225411)]

In [140]:
G.add_weighted_edges_from(list_of_edges)

In [141]:
# # Visualize Graph
nx.draw_networkx(G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [142]:
len(list(G.nodes))

627

In [145]:
# Corr. based on edges

In [146]:
# # 459 is corr!
sig_1 = final_data_df['1407']
# 619! 607!
list_of_node_crosscoef = []
for i in edge_series[1407]:
    sig_2 = final_data_df[str(i)]
#     sig2 = sig2[sig2.betriebsdatum == '11.01.16']
#     sig2.sort_values(by='soll_an_von', inplace=True)
#     sig2 = sig2.groupby(by='intervals', as_index=False).mean()
#     sig2_y = np.array(sig2.actual_travel_time_n_target)
    list_of_node_crosscoef.append((i, np.corrcoef(sig_1, sig_2)[0][1]))
zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()
plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

In [ ]:
# Save the gps data for further use

In [147]:
dict_gps = pd.Series(final.Mid_point_GPS_data.values,index=final.Route_Node_id).to_dict()
# print(len(dict_gps))
for i, v in dict_gps.items():
    if i not in list_of_nodes_Road_toKeep:
        del dict_gps[i]
dict_gps

627


{304: (47.37427574170736, 8.523350101018785),
 95: (47.37380262440648, 8.527598124871835),
 306: (47.37495215913542, 8.519264534466082),
 525: (47.3760938607128, 8.515596000196332),
 305: (47.37454861562561, 8.522749404904983),
 912: (47.37540930125573, 8.525079925856412),
 308: (47.37306093102224, 8.518697238051725),
 1578: (47.37096391163739, 8.51508411459711),
 197: (47.37067437372722, 8.521567166497919),
 1575: (47.370144980204614, 8.516530647460325),
 1027: (47.36778163059536, 8.520434309547642),
 196: (47.37134006341627, 8.52395520519553),
 420: (47.370674225659755, 8.52156949747705),
 153: (47.37194293521358, 8.525418341079343),
 195: (47.3719430910064, 8.525415801778253),
 96: (47.373037599567766, 8.528240430048324),
 711: (47.36934030110265, 8.533605442089872),
 100: (47.37290741425049, 8.532161776528572),
 142: (47.37098387208111, 8.536847691908568),
 97: (47.37485127109415, 8.531948580760604),
 1016: (47.377296475868924, 8.532224290619308),
 529: (47.37605952610066, 8.535849

In [149]:
# GPS data df
d = pd.DataFrame(dict_gps)
d.to_csv('gps_10min_demo.csv', index=False)

In [150]:
# Save Final DF as well
final_data_df.to_csv('signal_10min_demo.csv', index=False)

In [152]:
# Shape of final df
final_data_df.shape

(840, 627)

# Finding correlation coefficient

In [154]:
sig1 = final[final.Route_Node_id == 304]
sig1 = sig1[sig1.betriebsdatum == '07.01.16']
sig1.sort_values(by='soll_an_von', inplace=True)
# sig1 = sig1
sig1 = sig1.groupby(by='intervals', as_index=False).mean()
# sig1

In [155]:
sig2 = final[final.Route_Node_id == 306]
sig2 = sig2[sig2.betriebsdatum == '07.01.16']
sig2.sort_values(by='soll_an_von', inplace=True)
sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# sig2

In [156]:
sig1_X = np.array(sig1.intervals)
sig1_y = np.array(sig1.actual_travel_time_n_target)
sig1_y_zeromean = sig1_y - sig1_y.mean()

sig2_X = np.array(sig2.intervals)
sig2_y = np.array(sig2.actual_travel_time_n_target)
sig2_y_zeromean = sig2_y - sig2_y.mean()

plt.plot(sig1_X, sig1_y, sig2_X, sig2_y)

In [157]:
plt.plot(sig1_X, sig1_y_zeromean, sig2_X, sig2_y_zeromean)

In [159]:
from scipy.stats.stats import pearsonr
np.corrcoef(sig1_y_zeromean, sig2_y_zeromean)

array([[ 1.        , -0.06730167],
       [-0.06730167,  1.        ]])

In [160]:
pearsonr_coef = pearsonr(sig1_y, sig2_y)
pearsonr_coef

(-0.06730167127769561, 0.5430032484101102)

In [161]:
edge_series.value_counts()[:5]

339     82
1512    82
1367    82
1160    82
335     81
dtype: int64

In [163]:
%matplotlib
sig1 = np.array(final_data_df[str(1512)])

list_of_node_crosscoef = []

for i in edge_series[1512]:
    sig2 = np.array(final_data_df[str(i)])
    list_of_node_crosscoef.append((i, np.corrcoef(sig1, sig2)[0][1]))

zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()

plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

Using matplotlib backend: Qt5Agg


In [164]:
# # Autocorrelation
# def autocorr(x, t=1):
#     return np.corrcoef(np.array([x[:-t], x[t:]]))
# autocorr(sig2_y_zeromean)

array([[1.        , 0.05362642],
       [0.05362642, 1.        ]])

In [173]:
one_day = final_data_df[:84]
%matplotlib
import seaborn as sns
corr = one_day.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

Using matplotlib backend: Qt5Agg


In [181]:
final_data_df.corr().unstack().sort_values(ascending=False).drop_duplicates()[:20]

1581  1581    1.000000
649   699     0.972909
      399     0.961603
549   561     0.951361
399   699     0.943797
398   649     0.941521
      699     0.927616
88    399     0.925705
399   398     0.925450
649   88      0.893778
698   109     0.891008
699   88      0.879424
398   88      0.867680
159   906     0.837418
399   650     0.832998
554   561     0.816731
1328  1399    0.816101
141   710     0.806244
650   649     0.804341
      699     0.789882
dtype: float64

In [177]:
sig1 = final[final.Route_Node_id == 549]
sig1 = sig1[sig1.betriebsdatum == '07.01.16']
sig1.sort_values(by='soll_an_von', inplace=True)
# sig1 = sig1
sig1 = sig1.groupby(by='intervals', as_index=False).mean()
# sig1
sig2 = final[final.Route_Node_id == 561]
sig2 = sig2[sig2.betriebsdatum == '07.01.16']
sig2.sort_values(by='soll_an_von', inplace=True)
sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# sig2

sig1_X = np.array(sig1.intervals)
sig1_y = np.array(sig1.actual_travel_time_n_target)
sig1_y_zeromean = sig1_y - sig1_y.mean()

sig2_X = np.array(sig2.intervals)
sig2_y = np.array(sig2.actual_travel_time_n_target)
sig2_y_zeromean = sig2_y - sig2_y.mean()

plt.plot(sig1_X, sig1_y_zeromean, sig2_X, sig2_y_zeromean)

In [ ]:
# Save this graph

In [182]:
len(final_data_df.columns)

627

In [183]:
a = [int(i) for i in final_data_df.columns.values.tolist()]

adj_mat = nx.to_numpy_matrix(G, nodelist=a)

np.savetxt("PeMSD7_W_627_V1.csv", adj_mat, delimiter=",")

In [184]:
final_data_df.to_csv('PeMSD7_V_627_V1.csv', index=False)

# Create the graph another way! V2

In [186]:
Nodes = list(set(new_final.Route_Node_id))

In [187]:
Dict_List_halt_ids = pd.Series(new_final.Route_Node_id_tuple.values,index=new_final.Route_Node_id).to_dict()
len(Dict_List)

627

In [188]:
# Create the new graph:

# GPS is in Dict_List
Edges = []
for key, value in Dict_List_halt_ids.items():
    for key_2, value_2 in Dict_List_halt_ids.items():
        if key != key_2:
#             if value[0] == value_2[0] or value[0] == value_2[1] or value[1] == value_2[0] or value[1] == value_2[1]: 
            if value[0] == value_2[0] or value[1] == value_2[1]:    
                Edges.append((int(key), int(key_2), 10/(vincenty(Dict_List[int(key)], Dict_List[int(key_2)]).kilometers)))
    


C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # Remove the CWD from sys.path while we load stuff.


In [189]:
len(Edges)

1950

In [190]:
New_G = nx.Graph()
New_G.add_nodes_from(list_of_nodes_Road_toKeep)
New_G.add_weighted_edges_from(Edges)

In [192]:
len(list(New_G.nodes))

627

In [193]:
nx.draw_networkx(New_G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [194]:
set1 = set(list(New_G.node))
set2 = set(new_final.Route_Node_id)
set2 - set1

set()

In [195]:
a = [int(i) for i in final_data_df.columns.values.tolist()]

adj_mat = nx.to_numpy_matrix(New_G, nodelist=a)

np.savetxt("PeMSD7_W_627_V2.csv", adj_mat, delimiter=",")

# Remove nodes with no connections


In [196]:
len(New_G.nodes)

627

In [197]:
Connected_New_G = max(nx.connected_component_subgraphs(New_G), key=len)
nx.draw_networkx(Connected_New_G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
# set_1 = set(list(G.nodes))
# set_2 = set([int(i) for i in list(Connected_New_G.nodes)])
# set_3 = set_1.difference(set_2)

# # Nodes to drop
# nodes_drop_to_equalGraphs = list(set_3)

# len(nodes_drop_to_equalGraphs)

# some_G = G.subgraph([int(i) for i in list(Connected_New_G.nodes)])

# nx.draw_networkx(some_G)

# len(set(final.betriebsdatum))

# # Save Files!
# Connected_final_df.shape[1]

# a = [int(i) for i in Connected_final_df.columns.values.tolist()]

# adj_mat = nx.to_numpy_matrix(some_G, nodelist=a)

# np.savetxt("PeMSD7_W_5min_V1.csv", adj_mat, delimiter=",")

# # Save the connected final df as well

# Connected_final_df.to_csv('final_5min_final_1months_233.csv')
# df = pd.read_csv('final_5min_final_1months_233.csv', nrows=252)
# # df

# d = pd.DataFrame(dict_gps)
# for i in nodes_drop_to_equalGraphs:
#     del d[i]
# d.to_csv('gps_10min_final_3months_233.csv')
# # d.columns.tolist() - [int(i) for i in Connected_final_df.columns.tolist()]
# def intersection(lst1, lst2): 
#     lst3 = [value for value in lst1 if value in lst2] 
#     return lst3
# len(intersection(d.columns.tolist(), [int(i) for i in Connected_final_df.columns.tolist()]))

In [231]:
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

import folium
import folium.plugins as plugins
import numpy as np

df = final_data_df

print(df.isna().sum().sum())

# gps data
df_2 = pd.read_csv('gps_10min_demo.csv')
# df_2 = d

df = (df - df.mean()) / (df.std() ** 1.2)

# df_2.drop('Unnamed: 0', axis=1, inplace=True)
# df.drop('Unnamed: 0', axis=1, inplace=True)

df = df[:336]

# Outer list:
list_df_gps = df_2.columns.values.tolist()
OuterList = []
for index, row in df.iterrows():
    Mid_List = []
    for j in list_df_gps:
#         print(j)
#         if (j not in df_2.columns.values.tolist()):
#             continue
        Time_List = df_2[j].values.tolist()
#         print(type(j))
#         print(row[0])
        Time_List.append(row[j])
        Mid_List.append(Time_List)
    OuterList.append(Mid_List)

m = folium.Map([47.36179377478453, 8.572671632662107], tiles='stamentoner', zoom_start=12)

list_of_id = df_2.values
list_of_id = list(zip(list_of_id[0], list_of_id[1]))

for j in list_of_id:
    folium.Circle(j, popup='<strong>Location One</strong>', radius=20).add_to(m)

index = [str(timedelta(seconds=i * time_period + starting_time)) for i in set(final.intervals)] * int(df.shape[0]/len(set(final.intervals)))

hm = plugins.HeatMapWithTime(OuterList, index=index)

hm.add_to(m)

m.save('HeatMap_moving.html')

0


# Save Files

In [ ]:
# a = [int(i) for i in final_data_df.columns.values.tolist()]

# adj_mat = nx.to_numpy_matrix(New_G, nodelist=a)

In [ ]:
# Connected_final_df.to_csv('PeMSD7_V_233.csv', header=False, index=False)

In [ ]:
# np.savetxt("PeMSD7_W_233_V2.csv", adj_mat, delimiter=",")

In [ ]:
# len(list(New_G.nodes))

In [ ]:
# New_G.edges(data=True)

In [ ]:
# Connected_final_df.shape